In [0]:
from pyspark.sql.functions import sum as _sum, avg, min, max

df_consumption = spark.table("silver_consumption_summary")

df_fact_consumption = (
    df_consumption
    .groupBy("consumption_region", "category")
    .agg(
        _sum("Total_kWh").alias("total_kWh"),
        avg("Peak_kWh").alias("avg_peak_kWh"),
        min("Min_kWh").alias("min_kWh")
    )
)

df_fact_consumption.write \
    .mode("overwrite") \
    .format("delta") \
    .save("abfss://sedpcontainer@sedpstorageaccount.dfs.core.windows.net/Gold/gold_fact_consumption_summary_delta")

spark.sql("""
    CREATE TABLE IF NOT EXISTS gold_fact_consumption_summary
    USING DELTA
    LOCATION 'abfss://sedpcontainer@sedpstorageaccount.dfs.core.windows.net/Gold/gold_fact_consumption_summary_delta'
""")

DataFrame[]